<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Define-helper-functions" data-toc-modified-id="Define-helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define helper functions</a></span></li><li><span><a href="#Load-and-process-data" data-toc-modified-id="Load-and-process-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load and process data</a></span><ul class="toc-item"><li><span><a href="#Financial-Serecy-Index" data-toc-modified-id="Financial-Serecy-Index-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Financial Serecy Index</a></span></li><li><span><a href="#PSC-snapshot-data" data-toc-modified-id="PSC-snapshot-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>PSC snapshot data</a></span></li><li><span><a href="#Active-companies-data" data-toc-modified-id="Active-companies-data-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Active companies data</a></span></li><li><span><a href="#Officers-data" data-toc-modified-id="Officers-data-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Officers data</a></span></li><li><span><a href="#Politicans-data" data-toc-modified-id="Politicans-data-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Politicans data</a></span></li><li><span><a href="#Disqualified-directors" data-toc-modified-id="Disqualified-directors-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Disqualified directors</a></span></li></ul></li><li><span><a href="#Output-to-CSV-files-for-use-in-analysis-and-network-database" data-toc-modified-id="Output-to-CSV-files-for-use-in-analysis-and-network-database-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Output to CSV files for use in analysis and network database</a></span></li></ul></div>

# Import libraries

In [1]:
##Load in libraries
import pandas as pd
import numpy as np
import json
import re
import datetime
import requests as r
from dateutil.relativedelta import relativedelta
from pandas.io.json import json_normalize
import seaborn as sns
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', 500)
%matplotlib inline

# Define helper functions

In [ ]:
def secret_function(x):
    if x['country_of_residence_normal'] in secret_jurisdictions or x['address_country_normal'] in secret_jurisdictions or x['registered_country_normal'] in secret_jurisdictions:
        return True
    else:
        return False
def non_rle_function(x):
    if x['kind'] == 'corporate-entity-person-with-significant-control' and ~pd.isnull(x['registered_country_normal']) and (x['address_country_normal'] not in rec_list and x['registered_country_normal'] not in rec_list):
        return True
    else:
        return False
def secret_officer_function(x):
    if x['country_of_residence_normal'] in secret_jurisdictions or x['address_country_normal'] in secret_jurisdictions:
        return True
    else:
        return False
def date_check(date):
    if re.match(('^\d{4}-\d{2}-\d{2}$'), date): return 'full date'
    if re.match(('^\d{4}-\d{2}$'), date): return 'year and month'
    if re.match(('^\d{4}$'), date): return 'year only'
    return ('unknown format')
def flatten_legislatures(legislatures):
    if isinstance(legislatures, list):
        return pd.Series({'legislatures.popolo_url': legis.get('popolo_url', 'blank') for legis in legislatures})
    else:
        return pd.Series({})
def flatten_identifiers(identifiers):
    if isinstance(identifiers, list):
        return pd.Series({'identifier.{}'.format(ident.get('scheme', 'blank')): ident.get('identifier') for ident in identifiers})
    else:
        return pd.Series({})
def flatten_identifiers(identifiers):
    if isinstance(identifiers, list):
        return pd.Series({'identifier.{}'.format(ident.get('scheme', 'blank')): ident.get('identifier') for ident in identifiers})
    else:
        return pd.Series({})
def flatten_contacts(contact_details):
    if isinstance(contact_details, list):
        return pd.Series({'contact_details.{}'.format(contact.get('type', 'blank')): contact.get('value', '') for contact in contact_details})
        #return #pd.Series({'contact_details.{}'.format(contact.get('type', 'blank')): contact.get('value', '')for contact in contact_details})
    else:
        return pd.Series({})
def load_every_country_json(url):
    """Function that takes an everypolitician.org JSON input and gives a flat dataframe"""
    json_data = r.get(url).json()
    df_country = pd.DataFrame(json_data)

    # Flatten the contact details
    df2 = df_country.merge(df_country.legislatures.apply(flatten_legislatures),
                   left_index=True,
                   right_index=True).drop('legislatures', axis=1)
    return df2
def process_url(url):
    res = r.get(url)
    df_persons = pd.DataFrame(res.json().get('persons'))
    
    try:
        # Flatten the contact details
        df2 = df_persons.merge(df_persons.contact_details.apply(flatten_contacts), 
                       left_index=True, 
                       right_index=True).drop('contact_details', axis=1)
    except:
        pass
    
    try:
        # Flatten the identifier details
        df3 = df2.merge(df2.identifiers.apply(flatten_identifiers), 
                        left_index=True, 
                        right_index=True).drop('identifiers', axis=1)
    except:
        pass
    
    try:
        df3['date_format'] = df3['birth_date'].astype(str).apply(date_check)
    except:
        return pd.DataFrame()

    df_subset = df3[df3['date_format'].isin(['full date', 'year and month'])]
    return df_subset

# Load and process data

## Financial Serecy Index

This data is sourced from the Tax Justice Initiative *Financial Secrecy Index 2018* which is available [here](https://www.financialsecrecyindex.com/).

In [2]:
##Load data
secrecy_index = pd.read_csv('data/original_data/financial_secrecy_index_2018.csv')
secrecy_index['normalized_jurisdiction'] = secrecy_index.Jurisdiction.str.upper()
##Secrecy jurisdictions are those with secrecy socres of 60 and above and not USA
secret_jurisdictions_series = secrecy_index[(secrecy_index['Secrecy Score'] >= 60) & (secrecy_index.normalized_jurisdiction != 'USA')]['normalized_jurisdiction']
secret_jurisdictions = secret_jurisdictions_series.tolist()

## PSC snapshot data

PSC data is accurate as of 2018-3-1. It is sourced from Companies House. The most recent version is available to download [here](http://download.companieshouse.gov.uk/en_pscdata.html).

In [33]:
original_psc_data = pd.read_json('data/original_data/persons-with-significant-control-snapshot-2018-03-01.txt',lines=True)
all_records_psc = pd.concat([original_psc_data['company_number'],json_normalize(original_psc_data['data'])],axis=1)
##Remove last line of DataFrame which is not a record
all_records_psc = all_records_psc.iloc[:-1].copy()
##Remove summary totals before of PSC not including statemnts
all_records_psc = all_records_psc[all_records_psc.kind != 'totals#persons-of-significant-control-snapshot']
##Create a DataFrame of ways of controlling companies. Other methods are inefficient, so best to use JSON normalize.
temp_df = all_records_psc[['company_number','natures_of_control']].dropna(subset=['natures_of_control'])
##Expensive operation, write to CSV to save time. Need to re-generate when PSC file updated
list_of_lists = []
for index,row in temp_df.iterrows():
    for item in row['natures_of_control']:
        list_of_lists.append([row['company_number'],item])
psc_controls = pd.DataFrame(list_of_lists)
psc_controls.columns = ['company_number','nature_of_control']
##Create additional columns
all_records_psc['month_year_birth'] = all_records_psc['date_of_birth.month'].fillna('').astype(str).str.replace('\.0','') + '-' + all_records_psc['date_of_birth.year'].fillna('').astype(str).str.replace('\.0','')
all_records_psc['month_year_birth'] = pd.to_datetime(all_records_psc['month_year_birth'],format='%m-%Y',errors='coerce')
all_records_psc['join_id'] = all_records_psc['name_elements.forename'].str.upper() + '_' + all_records_psc['name_elements.surname'].str.upper() + '_' + all_records_psc['month_year_birth'].astype(str)
##Create DataFrames for statment records and PSC records. Records DF is for where a PSC has been filed, statements DF is for
##statements that have been made by a company that do not involve actually disclosing a PSC e.g. no PSC.
psc_records = all_records_psc[pd.isnull(all_records_psc.statement)].copy()
psc_statements = all_records_psc[~ pd.isnull(all_records_psc.statement)].copy()
##Add in new field to mark any company or individual as likely based or incorporated in a secrecy jurisdiction
psc_records['address_country_normal'] = psc_records['address.country'].str.upper()
psc_records['address_country_normal'] = psc_records['address_country_normal'].fillna('')
psc_records['registered_country_normal'] = psc_records['identification.country_registered'].str.upper()
psc_records['registered_country_normal'] = psc_records['registered_country_normal'].fillna('')
psc_records['country_of_residence_normal'] = psc_records['country_of_residence'].str.upper()
psc_records['country_of_residence_normal'] = psc_records['country_of_residence_normal'].fillna('')
registered_country_clean_map = pd.read_csv('data/original_data/registered_country_cleaner_map.csv')
registered_country_clean_map.drop_duplicates(inplace=True)
registered_country_clean_map = pd.Series(registered_country_clean_map.clean.values,index=registered_country_clean_map.original)
psc_records['registered_country_normal'] = psc_records['registered_country_normal'].map(registered_country_clean_map)
address_country_clean_map = pd.read_csv('data/original_data/address_country_cleaner_map.csv')
address_country_clean_map.drop_duplicates(inplace=True)
address_country_clean_map = pd.Series(address_country_clean_map.clean.values,index=address_country_clean_map.original)
psc_records['address_country_normal'] = psc_records['address_country_normal'].map(address_country_clean_map)
psc_records['secret_base'] = psc_records.apply(secret_function,axis=1)
rec = pd.read_csv('data/original_data/recognised_stock_exchange_countries.csv')
rec_list = rec['country_name'].str.upper().tolist()
rec_list.extend(['ENGLAND','SCOTLAND','NORTHERN IRELAND','GREAT BRITAIN','UK','WALES','UNITED STATES OF AMERICA','UNITED STATES','ENGLAND & WALES',\
                'REPUBLIC OF IRELAND','IRELAND','ENGLAND AND WALES'])
psc_records['non_rle_country'] = psc_records.apply(non_rle_function,axis=1)

## Active companies data

Active companies data is accurate as of 2018-3-1. It is sourced from the Companies House, the most recent version is available [here](http://download.companieshouse.gov.uk/en_output.html) as the Free Company Data product.

In [11]:
##Load in Companies House Free data product
active_companies = pd.read_csv('data/original_data/BasicCompanyDataAsOneFile-2018-03-01.csv')
##Take leading whitespace out of column names
active_companies.columns = [x.strip() for x in active_companies.columns]
##Create column which combines first line of address and postcode for easier analysis
active_companies['first_and_postcode'] = active_companies['RegAddress.AddressLine1'] + '-' + active_companies['RegAddress.PostCode']
##Use Companies House 10 year file of dissolved and active companies from October 2017 (check date)
all_companies = pd.read_csv('data/original_data/live_comps_and_dissolved_10yrs.csv')
all_companies.columns = [x.strip() for x in all_companies.columns]
all_companies['first_and_postcode'] = all_companies['RegAddress.AddressLine1'] + '-' + all_companies['RegAddress.PostCode']
dissolved_companies = all_companies[~ pd.isnull(all_companies['DissolutionDate'])]

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,7,8,27,28,29,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Officers data

Officers data is accurate as of about March 2018 and sourced from OpenCorporates.com in bulk. Contact OpenCorporates for more information on how to acquire this data.

In [ ]:
all_officers = pd.read_csv('data/original_data/officers.csv',dtype={'id': int,'company_number': str,\
            'jurisdiction_code': str,'name': str ,'title': str, 'first_name': str , 'last_name': str,\
                'position': str, 'start_date': str, 'person_number': str, 'person_uid': str, 'end_date': str, 'current_status': str,\
                    'occupation': str, 'nationality': str, 'country_of_residence': str, 'partial_date_of_birth': str, 'type': str,\
                        'address.in_full': str, 'address.street_address': str, 'address.locality': str,\
                            'address.region,address.postal_code': str,'address.country': str, 'retrieved_at': str, 'source_url': str})
all_officers['country_of_residence_normal'] = all_officers['country_of_residence'].str.upper()  
all_officers['address_country_normal'] = all_officers['address.country'].str.upper()  
all_officers['secret_base'] = all_officers.apply(secret_officer_function,axis=1)

active_officers = (all_officers[pd.isnull(all_officers.end_date)]).copy()

active_officers.first_name = active_officers.first_name.fillna('')
active_officers['first_name_single'] = active_officers.first_name.apply(lambda x: x.split(' ')[0] if len(x) > 0 else '')

active_officers['join_id'] =  active_officers.first_name_single.str.upper() + '_' + active_officers.last_name.str.upper() + '_' + active_officers.partial_date_of_birth.fillna('') + '-01'

## Politicans data

Data on politicians is updated each time the script is run. The data is avaialble from [EveryPolitician.org](http://everypolitician.org/countries.html).

In [ ]:
print('Last run: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
df = load_every_country_json('https://raw.githubusercontent.com/everypolitician/everypolitician-data/master/countries.json')
df_full = pd.DataFrame(columns=['birth_date'])
for index, row in df.iterrows():
    df_persons = process_url(row['legislatures.popolo_url'])
    df_persons['country'] = row['country']
    df_full = pd.concat([df_full, df_persons])

Last run: 2018-03-28 13:27:18


In [ ]:
df_full['birth_date'] = pd.to_datetime(df_full['birth_date'],format='%Y-%m-%d',exact=True,errors='coerce')
df_full['year'] = df_full.birth_date.apply(lambda x: str(x.year))
df_full['month'] = df_full.birth_date.apply(lambda x: str(x.month))
df_full['month_year'] = df_full.birth_date.apply(lambda x: str(x.year)) + '-' + df_full.birth_date.apply(lambda x: str(x.month)) + '-01'
df_full['first_name'] = df_full.name.apply(lambda x: x.split(' ')[0] if len(x.split(' ')) > 1 else '')
df_full['last_name'] = df_full.name.apply(lambda x: x.split(' ')[-1] if len(x.split(' ')) > 1 else '')
df_full['join_id'] = df_full.first_name.str.upper() + '_' + df_full.last_name.str.upper() + '_' + df_full['month_year']

## Disqualified directors

This data was sourced from [OpenSanctions.org](https://www.opensanctions.org/) on 2018-3-28.

In [ ]:
d_directors = pd.read_json('data/original_data/gb-coh-disqualified.ijson',lines=True)
d_directors = d_directors.dropna(subset=['birth_dates'])
d_directors['birth_date_formatted'] = pd.to_datetime(d_directors.birth_dates.apply(lambda x: x[0]['date']))
d_directors['year'] = d_directors.birth_date_formatted.apply(lambda x: str(x.year))
d_directors['month'] = d_directors.birth_date_formatted.apply(lambda x: str(x.month))
d_directors['month_year'] = d_directors.birth_date_formatted.apply(lambda x: str(x.year)) + '-' + d_directors.birth_date_formatted.apply(lambda x: str(x.month)) + '-01'
d_directors['join_id'] = d_directors.first_name.str.upper() + '_' + d_directors.last_name.str.upper() + '_' + d_directors['month_year']
d_directors.to_csv('data/outputs/disqualified_directors.csv',index=False)

# Output to CSV files for use in analysis and network database 

In [ ]:
active_companies.to_csv('data/outputs/active_companies.csv',index=False)
psc_records.to_csv('data/outputs/psc_records.csv',index=False)
psc_statements.to_csv('data/outputs/psc_statements.csv',index=False)
psc_controls.to_csv('data/outputs/psc_controls.csv',index=False)
secret_jurisdictions_series.to_csv('data/outputs/secret_jurisdictions.csv',index=False)
active_officers.to_csv('data/outputs/active_officers.csv',index=False)
dissolved_companies.to_csv('data/outputs/dissolved_companies.csv',index=False)
df_full.to_csv('data/outputs/every_politician.csv',index=False)
d_directors.to_csv('data/outputs/disqualified_directors.csv',index=False)